In [3]:
from virusPropagationModel import *
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

In [4]:
model = ModeledPopulatedWorld(100,400,5)

In [5]:
simulation1 = Simulation(model,100)
simulation1.head()


AttributeError: 'Simulation' object has no attribute 'head'

In [ ]:
#for x in ['S','I']:
S =[(t,len(res[(res['time']==t) & (res['status']=='S')])) for t in res['time'].unique() ]
I =[(t,len(res[(res['time']==t) & (res['status']=='I')])) for t in res['time'].unique() ]
R =[(t,len(res[(res['time']==t) & (res['status']=='R')])) for t in res['time'].unique() ]
D =[(t,len(res[(res['time']==t) & (res['status']=='D')])) for t in res['time'].unique() ]
    

In [ ]:
dfS = pd.DataFrame(S, columns=['time', 'S'])
plt.plot(dfS['time'].values, dfS['S'].values, label = 'Susceptible')
dfI = pd.DataFrame(I, columns=['time', 'I'])
plt.plot(dfI['time'].values, dfI['I'].values, label = 'Infected')
dfR = pd.DataFrame(R, columns=['time', 'R'])
plt.plot(dfI['time'].values, dfR['R'].values, label = 'Recovered')
dfD = pd.DataFrame(D, columns=['time', 'D'])
plt.plot(dfD['time'].values, dfD['D'].values, label = 'Dead')

plt.plot(dfI['time'].values, dfR['R'].values+dfI['I'].values+dfS['S'].values + dfD['D'].values, label = 'Sum')
plt.title('SecondPlot CoronaABM')
plt.legend()
plt.show()
# plt.savefig('CoronaPlot.png')


In [ ]:
res[res['h_ID']==1].plot('time','loc')
plt.title('schedule')

In [ ]:
model.timecourse

In [ ]:
res = model.simulate(100)
res